### Imports

In [1]:
import pickle as pkl
import lxml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras
import importlib
import time
import traceback
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import random

import subprocess
import scipy
import seaborn as sns

from sklearn.utils import shuffle


from tensorflow.keras.datasets import imdb
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.cluster import SpectralClustering
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from ast import literal_eval
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Conv1D, Flatten, Reshape, MaxPooling1D,\
                        AveragePooling1D, Concatenate, LeakyReLU, Embedding,\
                        GlobalMaxPooling1D,GlobalAveragePooling1D,GaussianNoise,BatchNormalization,Add,ZeroPadding1D, Layer, Lambda
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.backend import int_shape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from pylab import *
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, average_precision_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.utils import shuffle



plt.rcParams['figure.dpi'] = 300

importlib.reload(tensorflow)

/miniconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/miniconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/miniconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/miniconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

<module 'tensorflow' from '/miniconda/lib/python3.6/site-packages/tensorflow/__init__.py'>

In [2]:
tqdm.pandas()

### Helper Fucntions

In [3]:
def create_interaction_dicts(interactions):

    drug_dict = {}
    target_dict = {}

    for i in tqdm(range(len(interactions))):

        drug_id = interactions['InChiKey'].values[i]
        target_id = interactions['target_aa_code'].values[i]
        binding = interactions['Y'].values[i]

        try:
            drug_dict[drug_id]
        except:
            drug_dict[drug_id] = {}

        try:
            drug_dict[drug_id][binding].append(target_id)
        except:
            drug_dict[drug_id][binding] = [target_id]

        try:
            target_dict[target_id]
        except:
            target_dict[target_id] = {}

        try:
            target_dict[target_id][binding].append(drug_id)
        except:
            target_dict[target_id][binding] = [drug_id]

    return drug_dict, target_dict

In [4]:
def create_adjacency(drug_dict, target_dict, full = True, include_negative = False):
    
    # Create Adjascency Matrix For Drugs x Amino Acids
    drug_list = list(drug_dict.keys())
    target_list = list(target_dict.keys())
    number_of_drugs = len(list(drug_dict.keys()))
    number_of_targets = len(list(target_dict.keys()))

    adjascency_matrix = np.zeros((number_of_drugs, number_of_targets))

    if include_negative == False: 
        for i in tqdm(range(number_of_drugs)):
            for j in range(number_of_targets):

                try:
                    if target_list[j] in drug_dict[drug_list[i]][1]:
                        adjascency_matrix[i][j] = 1
                except: 
                    None

                try: 
                    if target_list[j] in drug_dict[drug_list[i]][0]:
                        adjascency_matrix[i][j] = 0
                except: 
                    None
                    
    else : 
        for i in tqdm(range(number_of_drugs)):
            for j in range(number_of_targets):

                try:
                    if target_list[j] in drug_dict[drug_list[i]][1]:
                        adjascency_matrix[i][j] = 1
                except: 
                    None

                try: 
                    if target_list[j] in drug_dict[drug_list[i]][0]:
                        adjascency_matrix[i][j] = -1
                except: 
                    None
                
    if full == False: 
        return adjascency_matrix
    
    else: 
        true_adjacency_matrix_bipartite = np.block([
            [np.zeros((len(drug_dict), len(drug_dict))), adjascency_matrix],
            [adjascency_matrix.T, np.zeros((len(target_dict), len(target_dict)))]
        ])
        return true_adjacency_matrix_bipartite

In [5]:
def get_eigen(adjacency):
    
    start_time = time.time()
    eigen_values, eigen_vectors = np.linalg.eig(adjacency)
    end_time = time.time()
    print ("Time Taken : ", end_time - start_time, "s")
    eigen_values_sorted = np.sort(eigen_values)
    
    return eigen_values, eigen_vectors, eigen_values_sorted

In [6]:
def getNodeName(idx):
    # drugs then targets
    if idx > len(drug_dict_4):
        target_idx = idx - len(drug_dict_4)
        aa_code = list(target_dict_4.keys())[target_idx]
        ret = "Targ AA:"+aa_code[0:min(10,len(aa_code))]
    else:
        drug_idx = idx 
        ichikey_code = list(drug_dict_4.keys())[drug_idx]
        ret = "Chem IK:"+ichikey_code[0:min(10,len(ichikey_code))]
    return ret

### Data

In [ ]:
interactions_4 = pd.read_csv('/data/sars-busters/interactions/train_interactions_04.csv')
drug_dict_4, target_dict_4 = create_interaction_dicts(interactions_4)
adjacency_4 = create_adjacency(drug_dict_4, target_dict_4)
adjacency_4_neg = create_adjacency(drug_dict_4, target_dict_4, include_negative = True)
degree_4 = np.sum(adjacency_4, axis = 0) + 1e-2
degree_4_neg = np.sum(np.abs(adjacency_4_neg), axis = 0) + 1e-2
degree_ratio_4 = degree_4 / degree_4_neg

  0%|          | 0/62949 [00:00<?, ?it/s]

  0%|          | 0/7400 [00:00<?, ?it/s]

In [ ]:
interactions_8 = pd.read_csv('/data/sars-busters/interactions/interactions_08.csv')
drug_dict_8, target_dict_8 = create_interaction_dicts(interactions_8)
adjacency_8 = create_adjacency(drug_dict_8, target_dict_8)
adjacency_8_neg = create_adjacency(drug_dict_8, target_dict_8, include_negative = True)
degree_8 = np.sum(adjacency_8, axis = 0) + 1e-2
degree_8_neg = np.sum(np.abs(adjacency_8_neg), axis = 0) + 1e-2
degree_ratio_8 = degree_8 / degree_8_neg

In [ ]:
#interactions_dataset1 = pd.read_csv('/data/sars-busters-consolidated/nw_files/WholeNetworkEdgesTable.csv')
#dataset1_interactions.drop_duplicates(subset=['InChiKey','target_aa_code'])
#dataset1_interactions['Y'].value_counts()

In [ ]:
interactions_2 = pd.read_csv('/data/sars-busters-consolidated/interactions/dataset_2_unfiltered.csv')
interactions_2 = interactions_2.drop_duplicates(subset=['TargetLabel','SourceLabel'])
print(interactions_2['Y'].value_counts())

interactions_2 = interactions_2.rename(columns = {'TargetLabel':'target_aa_code','SourceLabel':'InChiKey'})


### Scatter Plots

#### Interaction 4

##### Hop Lengths

In [ ]:
num_nodes = adjacency_4.shape[0]
max_hop = 13
higher_adjacency_matrix = np.zeros((num_nodes, num_nodes, max_hop + 1))

higher_adjacency_matrix[:,:,0] =  np.identity(num_nodes)

for i in tqdm(range(1,max_hop)):
    higher_adjacency_matrix[:,:,i] = np.dot(higher_adjacency_matrix[:,:,i-1],adjacency_4)
higher_adjacency_matrix[:,:,max_hop] = np.ones((num_nodes, num_nodes))     

In [ ]:
path_length = (higher_adjacency_matrix != 0).argmax(axis = 2)

In [ ]:
plt.title("Path Lengths")
plt.xlabel("Hop Distance")
plt.ylabel("Frequency")
plt.hist(path_length.flatten(),bins=list(range(0,14)))
plt.show()

##### Eigen Decomposition

In [ ]:
eigen_values_4, eigen_vectors_4, eigen_values_sorted_4 = get_eigen(adjacency_4)

In [ ]:
# Transpose - Rows are now the eigen vectors 
eigen_vectors_4 = eigen_vectors_4.T

In [ ]:
print ("Maximum Eigen Value : ", max([x.real for x in eigen_values_4]))

In [ ]:
plt.title("EigenValue Distribution - BiPartite")
plt.xlabel("EigenValues")
plt.ylabel("Frequency")
plt.hist(eigen_values_4, bins = np.arange(-10, 10, 0.5))
plt.show()

##### SVD 

In [ ]:
adjacency_4_rect = create_adjacency(drug_dict_4, target_dict_4, full = False)

In [ ]:
U, S, V_h = scipy.linalg.svd(adjacency_4_rect, full_matrices = True)

In [ ]:
print ("Rectangualr Adjacency : ", adjacency_4_rect.shape)
print ("U :", U.shape)
print ("S :", S.shape)
print ("V_h :", V_h.shape)

In [ ]:
sanity_check = U @ np.vstack((np.diag(S), np.zeros((U.shape[0] - S.shape[0], S.shape[0])))) @ V_h

In [ ]:
sanity_check_diff = adjacency_4_rect - test

In [ ]:
max(sanity_check_diff.flatten())

In [ ]:
plt.title("Histogram Of Singular Values")
plt.hist(S, bins = 100)
plt.show()

In [ ]:
# import matplotlib as mpl
plt.rcParams['figure.dpi'] = 300

###### Drugs

In [ ]:
n_drugs = 10
drug_idx_start = 5
n_eigvals = 5

balanced_nodes = np.where((degree_ratio_4[7400:] > 0.3) & (degree_ratio_4[7400:] < 0.7))[0] + 7400
balanced_nodes_degree_4_neg = degree_4_neg[list(balanced_nodes)]
top_nodes_sub_idx = np.flip(np.argsort(balanced_nodes_degree_4_neg))[0:n_drugs]
top_nodes_idx = balanced_nodes[list(top_nodes_sub_idx)]

color_front = np.zeros((n_drugs, len(U)), dtype = object)
color_background = np.zeros((n_drugs, len(U)), dtype = object)
size_neg = np.zeros((n_drugs, len(U)), dtype = object)
zoom = 0.05
rownames = []

for idx, node in tqdm(enumerate(top_nodes_idx)):
    rownames.append(getNodeName(node))
    path_length_color = []
    color_front[idx] = [(0,0,0,0.0) if x == 0
                      else (1, 0, 0, 1) if (x == -1 and path_length[node,idx_in] > 12)
                      else (1, 0, 1, 1) if (x == -1 and path_length[node,idx_in] <= 12 and path_length[node,idx_in] > 6)
                      else (0, 0, 1, 1) if (x == -1 and path_length[node,idx_in] <= 6 and path_length[node,idx_in] > -1)
                      else (0, 1, 0, 1)
                      for idx_in, x in enumerate(adjacency_4_neg[:7400, node])]
    color_background[idx] = [(0,0,0,0.1) if x == 0
                      else (1, 0, 0, 0) if (x == -1 and path_length[node,idx_in] > 12)
                      else (1, 0, 1, 0) if (x == -1 and path_length[node,idx_in] <= 12 and path_length[node,idx_in] > 6)
                      else (0, 0, 1, 0) if (x == -1 and path_length[node,idx_in] <= 6 and path_length[node,idx_in] > -1)
                      else (0, 1, 0, 0)
                      for idx_in, x in enumerate(adjacency_4_neg[:7400, node])]
    size_neg[idx] = [2.0 if x == 0 else 20.0 for x in adjacency_4_neg[:7400, node]]

fig, axs = plt.subplots(n_drugs,n_eigvals) 
fig.set_size_inches(6.5 * n_eigvals, n_drugs * 4)

for idx in range(n_drugs):
    for eveci in range(n_eigvals):
        axs[idx,eveci].scatter(np.clip(U[:, eveci], -zoom, zoom),
                               np.clip(U[:, eveci+1], -zoom, zoom),
                               c = color_front[idx],
                               s = list(size_neg[idx]),zorder=2)
        axs[idx,eveci].scatter(np.clip(U[:, eveci], -zoom, zoom),
                               np.clip(U[:, eveci+1], -zoom, zoom),
                               c = color_background[idx],
                               s = list(size_neg[idx]),zorder=1)
        axs[idx,eveci].set(xlabel = f"U {eveci}", ylabel = f"U {eveci+1}")

for ax, row in zip(axs[:,0], rownames):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - 5, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center')
        
plt.show()

In [ ]:
n_drugs = 10
drug_idx_start = 5
n_eigvals = 5

balanced_nodes = np.where((degree_ratio_4[7400:] > 0.3) & (degree_ratio_4[7400:] < 0.7))[0] + 7400
balanced_nodes_degree_4_neg = degree_4_neg[list(balanced_nodes)]
top_nodes_sub_idx = np.flip(np.argsort(balanced_nodes_degree_4_neg))[0:n_drugs]
top_nodes_idx = balanced_nodes[list(top_nodes_sub_idx)]

color_front = np.zeros((n_drugs, len(U)), dtype = object)
color_background = np.zeros((n_drugs, len(U)), dtype = object)
size_neg = np.zeros((n_drugs, len(U)), dtype = object)
zoom = 0.05
rownames = []

for idx, node in tqdm(enumerate(top_nodes_idx)):
    rownames.append(getNodeName(node))
    path_length_color = []
    color_front[idx] = [(0,0,0,0.0) if x == 0
                      else (1, 0, 0, 1) if (x == -1 and path_length[node,idx_in] > 12)
                      else (1, 0, 1, 1) if (x == -1 and path_length[node,idx_in] <= 12 and path_length[node,idx_in] > 6)
                      else (0, 0, 1, 1) if (x == -1 and path_length[node,idx_in] <= 6 and path_length[node,idx_in] > -1)
                      else (0, 1, 0, 1)
                      for idx_in, x in enumerate(adjacency_4_neg[:7400, node])]
    color_background[idx] = [(0,0,0,0.1) if x == 0
                      else (1, 0, 0, 0) if (x == -1 and path_length[node,idx_in] > 12)
                      else (1, 0, 1, 0) if (x == -1 and path_length[node,idx_in] <= 12 and path_length[node,idx_in] > 6)
                      else (0, 0, 1, 0) if (x == -1 and path_length[node,idx_in] <= 6 and path_length[node,idx_in] > -1)
                      else (0, 1, 0, 0)
                      for idx_in, x in enumerate(adjacency_4_neg[:7400, node])]
    size_neg[idx] = [2.0 if x == 0 else 20.0 for x in adjacency_4_neg[:7400, node]]

fig, axs = plt.subplots(n_drugs,n_eigvals) 
fig.set_size_inches(6.5 * n_eigvals, n_drugs * 4)

for idx in range(n_drugs):
    for eveci in range(n_eigvals):
        axs[idx,eveci].scatter(np.clip(U[:, eveci], -zoom, zoom),
                               np.clip(U[:, eveci+1], -zoom, zoom),
                               c = color_front[idx],
                               s = list(size_neg[idx]),zorder=2)
        axs[idx,eveci].scatter(np.clip(U[:, eveci], -zoom, zoom),
                               np.clip(U[:, eveci+1], -zoom, zoom),
                               c = color_background[idx],
                               s = list(size_neg[idx]),zorder=1)
        axs[idx,eveci].set(xlabel = f"U {eveci}", ylabel = f"U {eveci+1}")

for ax, row in zip(axs[:,0], rownames):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - pad, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center')
        
plt.show()

###### Targets

In [ ]:
n_drugs = 10
drug_idx_start = 5
n_eigvals = 5

balanced_nodes = np.where((degree_ratio_4[:7400] > 0.3) & (degree_ratio_4[:7400] < 0.7))[0]
balanced_nodes_degree_4_neg = degree_4_neg[list(balanced_nodes)]
top_nodes_sub_idx = np.flip(np.argsort(balanced_nodes_degree_4_neg))[0:n_drugs]
top_nodes_idx = balanced_nodes[list(top_nodes_sub_idx)]

color_front = np.zeros((n_drugs, len(V_h)), dtype = object)
color_background = np.zeros((n_drugs, len(V_h)), dtype = object)
size_neg = np.zeros((n_drugs, len(V_h)), dtype = object)
zoom = 0.05
rownames = []

for idx, node in tqdm(enumerate(top_nodes_idx)):
    rownames.append(getNodeName(node))
    path_length_color = []
    color_front[idx] = [(0,0,0,0.0) if x == 0
                      else (1, 0, 0, 1) if (x == -1 and path_length[node,7400 + idx_in] > 12)
                      else (1, 0, 1, 1) if (x == -1 and path_length[node,7400 + idx_in] <= 12 and path_length[node,7400 + idx_in] > 6)
                      else (0, 0, 1, 1) if (x == -1 and path_length[node,7400 + idx_in] <= 6 and path_length[node,7400 + idx_in] > -1)
                      else (0, 1, 0, 1)
                      for idx_in, x in enumerate(adjacency_4_neg[7400:, node])]
    color_background[idx] = [(0,0,0,0.1) if x == 0
                      else (1, 0, 0, 0) if (x == -1 and path_length[node,7400 + idx_in] > 12)
                      else (1, 0, 1, 0) if (x == -1 and path_length[node,7400 + idx_in] <= 12 and path_length[node,7400 + idx_in] > 6)
                      else (0, 0, 1, 0) if (x == -1 and path_length[node,7400 + idx_in] <= 6 and path_length[node,7400 + idx_in] > -1)
                      else (0, 1, 0, 0)
                      for idx_in, x in enumerate(adjacency_4_neg[7400:, node])]
    size_neg[idx] = [2.0 if x == 0 else 20.0 for x in adjacency_4_neg[7400:, node]]

fig, axs = plt.subplots(n_drugs,n_eigvals) 
fig.set_size_inches(6.5 * n_eigvals, n_drugs * 4)

for idx in range(n_drugs):
    for eveci in range(n_eigvals):
        axs[idx,eveci].scatter(np.clip(V_h[eveci, :], -zoom, zoom),
                               np.clip(V_h[eveci + 1, :], -zoom, zoom),
                               c = color_front[idx],
                               s = list(size_neg[idx]),zorder=2)
        axs[idx,eveci].scatter(np.clip(V_h[eveci, :], -zoom, zoom),
                               np.clip(V_h[eveci + 1, :], -zoom, zoom),
                               c = color_background[idx],
                               s = list(size_neg[idx]),zorder=1)
        axs[idx,eveci].set(xlabel = f"U {eveci}", ylabel = f"U {eveci+1}")

for ax, row in zip(axs[:,0], rownames):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - 5, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center')
        
plt.show()

In [ ]:
n_drugs = 10
drug_idx_start = 5
n_eigvals = 5

balanced_nodes = np.where((degree_ratio_4[:7400] > 0.3) & (degree_ratio_4[:7400] < 0.7))[0]
balanced_nodes_degree_4_neg = degree_4_neg[list(balanced_nodes)]
top_nodes_sub_idx = np.flip(np.argsort(balanced_nodes_degree_4_neg))[0:n_drugs]
top_nodes_idx = balanced_nodes[list(top_nodes_sub_idx)]

color_front = np.zeros((n_drugs, len(V_h)), dtype = object)
color_background = np.zeros((n_drugs, len(V_h)), dtype = object)
size_neg = np.zeros((n_drugs, len(V_h)), dtype = object)
zoom = 0.05
rownames = []

for idx, node in tqdm(enumerate(top_nodes_idx)):
    rownames.append(getNodeName(node))
    path_length_color = []
    color_front[idx] = [(0,0,0,0.0) if x == 0
                      else (1, 0, 0, 1) if (x == -1 and path_length[node,7400 + idx_in] > 12)
                      else (1, 0, 1, 1) if (x == -1 and path_length[node,7400 + idx_in] <= 12 and path_length[node,7400 + idx_in] > 6)
                      else (0, 0, 1, 1) if (x == -1 and path_length[node,7400 + idx_in] <= 6 and path_length[node,7400 + idx_in] > -1)
                      else (0, 1, 0, 1)
                      for idx_in, x in enumerate(adjacency_4_neg[7400:, node])]
    color_background[idx] = [(0,0,0,0.1) if x == 0
                      else (1, 0, 0, 0) if (x == -1 and path_length[node,7400 + idx_in] > 12)
                      else (1, 0, 1, 0) if (x == -1 and path_length[node,7400 + idx_in] <= 12 and path_length[node,7400 + idx_in] > 6)
                      else (0, 0, 1, 0) if (x == -1 and path_length[node,7400 + idx_in] <= 6 and path_length[node,7400 + idx_in] > -1)
                      else (0, 1, 0, 0)
                      for idx_in, x in enumerate(adjacency_4_neg[7400:, node])]
    size_neg[idx] = [2.0 if x == 0 else 20.0 for x in adjacency_4_neg[7400:, node]]

fig, axs = plt.subplots(n_drugs,n_eigvals) 
fig.set_size_inches(6.5 * n_eigvals, n_drugs * 4)

for idx in range(n_drugs):
    for eveci in range(n_eigvals):
        axs[idx,eveci].scatter(np.clip(V_h[eveci, :], -zoom, zoom),
                               np.clip(V_h[eveci + 1, :], -zoom, zoom),
                               c = color_front[idx],
                               s = list(size_neg[idx]),zorder=2)
        axs[idx,eveci].scatter(np.clip(V_h[eveci, :], -zoom, zoom),
                               np.clip(V_h[eveci + 1, :], -zoom, zoom),
                               c = color_background[idx],
                               s = list(size_neg[idx]),zorder=1)
        axs[idx,eveci].set(xlabel = f"U {eveci}", ylabel = f"U {eveci+1}")

for ax, row in zip(axs[:,0], rownames):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - pad, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center')
        
plt.show()

##### Eigen Spokes

In [ ]:
n_drugs = 10
drug_idx_start = 5
n_eigvals = 5

balanced_nodes = np.where((degree_ratio_4 > 0.3) & (degree_ratio_4 < 0.7))[0]
balanced_nodes_degree_4_neg = degree_4_neg[list(balanced_nodes)]
top_nodes_sub_idx = np.flip(np.argsort(balanced_nodes_degree_4_neg))[0:n_drugs]
top_nodes_idx = balanced_nodes[list(top_nodes_sub_idx)]

color_front = np.zeros((n_drugs, len(eigen_vectors_4)), dtype = object)
color_background = np.zeros((n_drugs, len(eigen_vectors_4)), dtype = object)
size_neg = np.zeros((n_drugs, len(eigen_vectors_4)), dtype = object)
zoom = 0.05
rownames = []

for idx, node in enumerate(top_nodes_idx):
    rownames.append(getNodeName(node))
    path_length_color = []
    color_front[idx] = [(0,0,0,0.0) if x == 0
                      else (1, 0, 0, 1) if (x == -1 and path_length[node,idx] > 12)
                      else (1, 0, 1, 1) if (x == -1 and path_length[node,idx] <= 12 and path_length[node,idx] > 6)
                      else (0, 0, 1, 1) if (x == -1 and path_length[node,idx] <= 6 and path_length[node,idx] > -1)
                      else (0, 1, 0, 1)
                      for idx, x in enumerate(adjacency_4_neg[:, node])]
    color_background[idx] = [(0, 0, 0, 0.1) if x == 0
                      else (1, 0, 0, 0) if (x == -1 and path_length[node,idx] > 12)
                      else (1, 0, 1, 0) if (x == -1 and path_length[node,idx] <= 12 and path_length[node,idx] > 6)
                      else (0, 0, 1, 0) if (x == -1 and path_length[node,idx] <= 6 and path_length[node,idx] > -1)
                      else (0, 1, 0, 0)
                      for idx, x in enumerate(adjacency_4_neg[:, node])]
    size_neg[idx] = [2.0 if x == 0 else 20.0 for x in adjacency_4_neg[:, node]]

fig, axs = plt.subplots(n_drugs,n_eigvals) 
fig.set_size_inches(6.5 * n_eigvals, n_drugs * 4)

pos_only = True

for idx in range(n_drugs):
    for eveci in range(n_eigvals):
        
        evecj = eveci + 1
        
        if pos_only: 
            evec_i = (2 * eveci) + 1
            evec_j = evec_i + 2
            
            
        axs[idx,eveci].scatter(np.clip(eigen_vectors_4[evec_i], -zoom, zoom),
                               np.clip(eigen_vectors_4[evec_j], -zoom, zoom),
                               c = color_front[idx],
                               s = list(size_neg[idx]),zorder=2)
        axs[idx,eveci].scatter(np.clip(eigen_vectors_4[evec_i], -zoom, zoom),
                               np.clip(eigen_vectors_4[evec_j], -zoom, zoom),
                               c = color_background[idx],
                               s = list(size_neg[idx]),zorder=1)
        axs[idx,eveci].set(xlabel = f"EigenVector {evec_i} | Lambda = {eigen_values_4[evec_i]:.2f}", ylabel = f"EigenVector {evec_j}")

for ax, row in zip(axs[:,0], rownames):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - 5, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center')
        
plt.show()

#### Interactions 8

In [ ]:
interactions_8['Y'].value_counts()

In [ ]:
adjacency_8.shape

##### Hop Length

In [ ]:
num_nodes = adjacency_8.shape[0]
max_hop = 13
higher_adjacency_matrix = np.zeros((num_nodes, num_nodes, max_hop + 1))

higher_adjacency_matrix[:,:,0] =  np.identity(num_nodes)

for i in tqdm(range(1,max_hop)):
    higher_adjacency_matrix[:,:,i] = np.dot(higher_adjacency_matrix[:,:,i-1],adjacency_8)
higher_adjacency_matrix[:,:,max_hop] = np.ones((num_nodes, num_nodes))     

In [ ]:
path_length = (higher_adjacency_matrix != 0).argmax(axis = 2)

In [ ]:
plt.title("Path Lengths")
plt.xlabel("Hop Distance")
plt.ylabel("Frequency")
plt.hist(path_length.flatten(),bins=list(range(0,14)))
plt.show()

##### Eigen Decomposition

In [ ]:
eigen_values_8, eigen_vectors_8, eigen_values_sorted_8 = get_eigen(adjacency_8)

In [ ]:
# Transpose - Rows are now the eigen vectors 
eigen_vectors_8 = eigen_vectors_8.T

In [ ]:
print ("Maximum Eigen Value : ", max([x.real for x in eigen_values_8]))

In [ ]:
plt.title("EigenValue Distribution - BiPartite")
plt.xlabel("EigenValues")
plt.ylabel("Frequency")
plt.hist(eigen_values_8, bins = np.arange(-10, 10, 0.5))
plt.show()

##### SVD

In [ ]:
adjacency_8_rect = create_adjacency(drug_dict_8, target_dict_8, full = False)
U, S, V_h = scipy.linalg.svd(adjacency_8_rect, full_matrices = True)
print ("Rectangualr Adjacency : ", adjacency_8_rect.shape)
print ("U :", U.shape)
print ("S :", S.shape)
print ("V_h :", V_h.shape)

In [ ]:
plt.title("Histogram Of Singular Values")
plt.hist(S, bins = 100)
plt.show()

In [ ]:
plt.title("Histogram Of Singular Values")
plt.hist(S, bins = 100)
plt.show()

###### Drugs

In [ ]:
n_drugs = 10
drug_idx_start = 5
n_eigvals = 5

drug_end = 7091

balanced_nodes = np.where((degree_ratio_8[drug_end:] > 0.3) & (degree_ratio_8[drug_end:] < 0.7))[0] + drug_end
balanced_nodes_degree_8_neg = degree_8_neg[list(balanced_nodes)]
top_nodes_sub_idx = np.flip(np.argsort(balanced_nodes_degree_8_neg))[0:n_drugs]
top_nodes_idx = balanced_nodes[list(top_nodes_sub_idx)]

color_front = np.zeros((n_drugs, len(U)), dtype = object)
color_background = np.zeros((n_drugs, len(U)), dtype = object)
size_neg = np.zeros((n_drugs, len(U)), dtype = object)
zoom = 0.05
rownames = []

for idx, node in tqdm(enumerate(top_nodes_idx)):
    rownames.append(getNodeName(node))
    path_length_color = []
    color_front[idx] = [(0,0,0,0.0) if x == 0
                      else (1, 0, 0, 1) if (x == -1 and path_length[node,idx_in] > 12)
                      else (1, 0, 1, 1) if (x == -1 and path_length[node,idx_in] <= 12 and path_length[node,idx_in] > 6)
                      else (0, 0, 1, 1) if (x == -1 and path_length[node,idx_in] <= 6 and path_length[node,idx_in] > -1)
                      else (0, 1, 0, 1)
                      for idx_in, x in enumerate(adjacency_8_neg[:drug_end, node])]
    color_background[idx] = [(0,0,0,0.1) if x == 0
                      else (1, 0, 0, 0) if (x == -1 and path_length[node,idx_in] > 12)
                      else (1, 0, 1, 0) if (x == -1 and path_length[node,idx_in] <= 12 and path_length[node,idx_in] > 6)
                      else (0, 0, 1, 0) if (x == -1 and path_length[node,idx_in] <= 6 and path_length[node,idx_in] > -1)
                      else (0, 1, 0, 0)
                      for idx_in, x in enumerate(adjacency_8_neg[:drug_end, node])]
    size_neg[idx] = [2.0 if x == 0 else 20.0 for x in adjacency_8_neg[:drug_end, node]]

fig, axs = plt.subplots(n_drugs,n_eigvals) 
fig.set_size_inches(6.5 * n_eigvals, n_drugs * 4)

for idx in range(n_drugs):
    for eveci in range(n_eigvals):
        axs[idx,eveci].scatter(np.clip(U[:, eveci], -zoom, zoom),
                               np.clip(U[:, eveci+1], -zoom, zoom),
                               c = color_front[idx],
                               s = list(size_neg[idx]),zorder=2)
        axs[idx,eveci].scatter(np.clip(U[:, eveci], -zoom, zoom),
                               np.clip(U[:, eveci+1], -zoom, zoom),
                               c = color_background[idx],
                               s = list(size_neg[idx]),zorder=1)
        axs[idx,eveci].set(xlabel = f"U {eveci}", ylabel = f"U {eveci+1}")

for ax, row in zip(axs[:,0], rownames):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - 5, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center')
        
plt.show()

In [ ]:
#Same as above, but all negative samples are red.  For a presentation
#New negative sampling 

n_drugs = 10
drug_idx_start = 5
n_eigvals = 5

drug_end = 7091

balanced_nodes = np.where((degree_ratio_8[drug_end:] > 0.3) & (degree_ratio_8[drug_end:] < 0.7))[0] + drug_end
balanced_nodes_degree_8_neg = degree_8_neg[list(balanced_nodes)]
top_nodes_sub_idx = np.flip(np.argsort(balanced_nodes_degree_8_neg))[0:n_drugs]
top_nodes_idx = balanced_nodes[list(top_nodes_sub_idx)]

color_front = np.zeros((n_drugs, len(U)), dtype = object)
color_background = np.zeros((n_drugs, len(U)), dtype = object)
size_neg = np.zeros((n_drugs, len(U)), dtype = object)
zoom = 0.05
rownames = []

for idx, node in tqdm(enumerate(top_nodes_idx)):
    rownames.append(getNodeName(node))
    path_length_color = []
    color_front[idx] = [(0,0,0,0.0) if x == 0
                      else (1, 0, 0, 1) if (x == -1 and path_length[node,idx_in] > 12)
                      else (1, 0, 0, 1) if (x == -1 and path_length[node,idx_in] <= 12 and path_length[node,idx_in] > 6)
                      else (1, 0, 0, 1) if (x == -1 and path_length[node,idx_in] <= 6 and path_length[node,idx_in] > -1)
                      else (0, 1, 0, 1)
                      for idx_in, x in enumerate(adjacency_8_neg[:drug_end, node])]
    color_background[idx] = [(0,0,0,0.1) if x == 0
                      else (1, 0, 0, 0) if (x == -1 and path_length[node,idx_in] > 12)
                      else (1, 0, 1, 0) if (x == -1 and path_length[node,idx_in] <= 12 and path_length[node,idx_in] > 6)
                      else (0, 0, 1, 0) if (x == -1 and path_length[node,idx_in] <= 6 and path_length[node,idx_in] > -1)
                      else (0, 1, 0, 0)
                      for idx_in, x in enumerate(adjacency_8_neg[:drug_end, node])]
    size_neg[idx] = [2.0 if x == 0 else 20.0 for x in adjacency_8_neg[:drug_end, node]]

fig, axs = plt.subplots(n_drugs,n_eigvals) 
fig.set_size_inches(6.5 * n_eigvals, n_drugs * 4)

for idx in range(n_drugs):
    for eveci in range(n_eigvals):
        axs[idx,eveci].scatter(np.clip(U[:, eveci], -zoom, zoom),
                               np.clip(U[:, eveci+1], -zoom, zoom),
                               c = color_front[idx],
                               s = list(size_neg[idx]),zorder=2)
        axs[idx,eveci].scatter(np.clip(U[:, eveci], -zoom, zoom),
                               np.clip(U[:, eveci+1], -zoom, zoom),
                               c = color_background[idx],
                               s = list(size_neg[idx]),zorder=1)
        axs[idx,eveci].set(xlabel = f"U {eveci}", ylabel = f"U {eveci+1}")

for ax, row in zip(axs[:,0], rownames):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - 5, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center')
        
plt.show()

In [ ]:
#Old (roughly) negative samples based on hop distance 

n_drugs = 5
drug_idx_start = 5
n_eigvals = 5

drug_end = 7091

balanced_nodes = np.where((degree_ratio_8[drug_end:] > 0.3) & (degree_ratio_8[drug_end:] < 0.7))[0] + drug_end
balanced_nodes_degree_8_neg = degree_8_neg[list(balanced_nodes)]
top_nodes_sub_idx = np.flip(np.argsort(balanced_nodes_degree_8_neg))[0:n_drugs]
top_nodes_idx = balanced_nodes[list(top_nodes_sub_idx)]

color_front = np.zeros((n_drugs, len(U)), dtype = object)
color_background = np.zeros((n_drugs, len(U)), dtype = object)
size_neg = np.zeros((n_drugs, len(U)), dtype = object)
zoom = 0.05
rownames = []

for idx, node in tqdm(enumerate(top_nodes_idx)):
    rownames.append(getNodeName(node))
    path_length_color = []
    color_front[idx] = [(1, 0, 0, 1) if (path_length[node,idx_in] > 8)
                        else (0, 1, 0, 1) if x == 1
                        else (0,0,0,0.0)
                        for idx_in, x in enumerate(adjacency_8_neg[:drug_end, node])]
    color_background[idx] = [(0,0,0,0.1) if x == 0
                      else (1, 0, 0, 0) if (x == -1 and path_length[node,idx_in] > 12)
                      else (1, 0, 1, 0) if (x == -1 and path_length[node,idx_in] <= 12 and path_length[node,idx_in] > 6)
                      else (0, 0, 1, 0) if (x == -1 and path_length[node,idx_in] <= 6 and path_length[node,idx_in] > -1)
                      else (0, 1, 0, 0)
                      for idx_in, x in enumerate(adjacency_8_neg[:drug_end, node])]
    size_neg[idx] = [30.0 if (path_length[node,idx_in] > 8)
                        else 2.0 if x == 0
                        else 30.0
                        for idx_in, x in enumerate(adjacency_8_neg[:drug_end, node])]

fig, axs = plt.subplots(n_drugs,n_eigvals) 
fig.set_size_inches(6.5 * n_eigvals, n_drugs * 4)

for idx in range(n_drugs):
    for eveci in range(n_eigvals):
        axs[idx,eveci].scatter(np.clip(U[:, eveci], -zoom, zoom),
                               np.clip(U[:, eveci+1], -zoom, zoom),
                               c = color_front[idx],
                               s = list(size_neg[idx]),zorder=2)
        axs[idx,eveci].scatter(np.clip(U[:, eveci], -zoom, zoom),
                               np.clip(U[:, eveci+1], -zoom, zoom),
                               c = color_background[idx],
                               s = list(size_neg[idx]),zorder=1)
        axs[idx,eveci].set(xlabel = f"U {eveci}", ylabel = f"U {eveci+1}")

for ax, row in zip(axs[:,0], rownames):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - 5, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center')
        
plt.show()

###### Targets

In [ ]:
n_drugs = 10
drug_idx_start = 5
n_eigvals = 5

drug_end = 7091

balanced_nodes = np.where((degree_ratio_8[:drug_end] > 0.3) & (degree_ratio_8[:drug_end] < 0.7))[0]
balanced_nodes_degree_8_neg = degree_8_neg[list(balanced_nodes)]
top_nodes_sub_idx = np.flip(np.argsort(balanced_nodes_degree_8_neg))[0:n_drugs]
top_nodes_idx = balanced_nodes[list(top_nodes_sub_idx)]

color_front = np.zeros((n_drugs, len(V_h)), dtype = object)
color_background = np.zeros((n_drugs, len(V_h)), dtype = object)
size_neg = np.zeros((n_drugs, len(V_h)), dtype = object)
zoom = 0.05
rownames = []

for idx, node in tqdm(enumerate(top_nodes_idx)):
    rownames.append(getNodeName(node))
    path_length_color = []
    color_front[idx] = [(0,0,0,0.0) if x == 0
                      else (1, 0, 0, 1) if (x == -1 and path_length[node,drug_end + idx_in] > 12)
                      else (1, 0, 1, 1) if (x == -1 and path_length[node,drug_end + idx_in] <= 12 and path_length[node,drug_end + idx_in] > 6)
                      else (0, 0, 1, 1) if (x == -1 and path_length[node,drug_end + idx_in] <= 6 and path_length[node,drug_end + idx_in] > -1)
                      else (0, 1, 0, 1)
                      for idx_in, x in enumerate(adjacency_8_neg[drug_end:, node])]
    color_background[idx] = [(0,0,0,0.1) if x == 0
                      else (1, 0, 0, 0) if (x == -1 and path_length[node,drug_end + idx_in] > 12)
                      else (1, 0, 1, 0) if (x == -1 and path_length[node,drug_end + idx_in] <= 12 and path_length[node,drug_end + idx_in] > 6)
                      else (0, 0, 1, 0) if (x == -1 and path_length[node,drug_end + idx_in] <= 6 and path_length[node,drug_end + idx_in] > -1)
                      else (0, 1, 0, 0)
                      for idx_in, x in enumerate(adjacency_8_neg[drug_end:, node])]
    size_neg[idx] = [2.0 if x == 0 else 20.0 for x in adjacency_8_neg[drug_end:, node]]

fig, axs = plt.subplots(n_drugs,n_eigvals) 
fig.set_size_inches(6.5 * n_eigvals, n_drugs * 4)

for idx in range(n_drugs):
    for eveci in range(n_eigvals):
        axs[idx,eveci].scatter(np.clip(V_h[eveci, :], -zoom, zoom),
                               np.clip(V_h[eveci + 1, :], -zoom, zoom),
                               c = color_front[idx],
                               s = list(size_neg[idx]),zorder=2)
        axs[idx,eveci].scatter(np.clip(V_h[eveci, :], -zoom, zoom),
                               np.clip(V_h[eveci + 1, :], -zoom, zoom),
                               c = color_background[idx],
                               s = list(size_neg[idx]),zorder=1)
        axs[idx,eveci].set(xlabel = f"U {eveci}", ylabel = f"U {eveci+1}")

for ax, row in zip(axs[:,0], rownames):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - 5, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center')
        
plt.show()

##### Eigen Spokes

In [ ]:
n_drugs = 10
drug_idx_start = 5
n_eigvals = 5

balanced_nodes = np.where((degree_ratio_8 > 0.3) & (degree_ratio_8 < 0.7))[0]
balanced_nodes_degree_8_neg = degree_8_neg[list(balanced_nodes)]
top_nodes_sub_idx = np.flip(np.argsort(balanced_nodes_degree_8_neg))[0:n_drugs]
top_nodes_idx = balanced_nodes[list(top_nodes_sub_idx)]

color_front = np.zeros((n_drugs, len(eigen_vectors_8)), dtype = object)
color_background = np.zeros((n_drugs, len(eigen_vectors_8)), dtype = object)
size_neg = np.zeros((n_drugs, len(eigen_vectors_8)), dtype = object)
zoom = 0.05
rownames = []

for idx, node in enumerate(top_nodes_idx):
    rownames.append(getNodeName(node))
    path_length_color = []
    color_front[idx] = [(0,0,0,0.0) if x == 0
                      else (1, 0, 0, 1) if (x == -1 and path_length[node,idx] > 12)
                      else (1, 0, 1, 1) if (x == -1 and path_length[node,idx] <= 12 and path_length[node,idx] > 6)
                      else (0, 0, 1, 1) if (x == -1 and path_length[node,idx] <= 6 and path_length[node,idx] > -1)
                      else (0, 1, 0, 1)
                      for idx, x in enumerate(adjacency_8_neg[:, node])]
    color_background[idx] = [(0, 0, 0, 0.1) if x == 0
                      else (1, 0, 0, 0) if (x == -1 and path_length[node,idx] > 12)
                      else (1, 0, 1, 0) if (x == -1 and path_length[node,idx] <= 12 and path_length[node,idx] > 6)
                      else (0, 0, 1, 0) if (x == -1 and path_length[node,idx] <= 6 and path_length[node,idx] > -1)
                      else (0, 1, 0, 0)
                      for idx, x in enumerate(adjacency_8_neg[:, node])]
    size_neg[idx] = [2.0 if x == 0 else 20.0 for x in adjacency_8_neg[:, node]]

fig, axs = plt.subplots(n_drugs,n_eigvals) 
fig.set_size_inches(6.5 * n_eigvals, n_drugs * 4)

pos_only = True

for idx in range(n_drugs):
    for eveci in range(n_eigvals):
        
        evecj = eveci + 1
        
        if pos_only:
            if eigen_values_8[0] > 0:
                evec_i = (2 * eveci)
                evec_j = evec_i + 2
            else :
                evec_i = (2 * eveci) + 1
                evec_j = evec_i + 2
            
            
        axs[idx,eveci].scatter(np.clip(eigen_vectors_8[evec_i], -zoom, zoom),
                               np.clip(eigen_vectors_8[evec_j], -zoom, zoom),
                               c = color_front[idx],
                               s = list(size_neg[idx]),zorder=2)
        axs[idx,eveci].scatter(np.clip(eigen_vectors_8[evec_i], -zoom, zoom),
                               np.clip(eigen_vectors_8[evec_j], -zoom, zoom),
                               c = color_background[idx],
                               s = list(size_neg[idx]),zorder=1)
        axs[idx,eveci].set(xlabel = f"EigenVector {evec_i} | Lambda = {eigen_values_8[evec_i]:.2f}", ylabel = f"EigenVector {evec_j}")

for ax, row in zip(axs[:,0], rownames):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - 5, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center')
        
plt.show()

#### Interactions 2

There is an issue with adjacency matrix and possible drug_dict.  So we use networkx.  We also use the full dataset and reselect the connected component.

In [ ]:
import networkx as nx

In [ ]:
interactions_pos = interactions_2[interactions_2['Y'] == 1]
nodes = set(interactions_pos['Target']).union(set(interactions_pos['Source']))
len(nodes)

In [ ]:
edges = [(row[1]['Target'],row[1]['Source']) for row in tqdm(interactions_pos.iterrows())]
G2 = nx.Graph(edges)

In [ ]:
cc = nx.connected_components(G2)
lens = [len(c) for c in cc]
print(sort(lens)[-10:])

In [ ]:
subs = list(nx.connected_component_subgraphs(G2))

In [ ]:
len(subs[0].nodes)

In [ ]:
bigG = subs[0]

In [ ]:
adj_mat = nx.adjacency_matrix(bigG).toarray()

In [ ]:
num_nodes = adj_mat.shape[0]
max_hop = 13
higher_adj_mat = np.zeros((num_nodes, num_nodes, max_hop + 1))

higher_adj_mat[:,:,0] =  np.identity(num_nodes)

for i in tqdm(range(1,max_hop)):
    higher_adj_mat[:,:,i] = np.dot(higher_adj_mat[:,:,i-1],adj_mat)
higher_adj_mat[:,:,max_hop] = np.ones((num_nodes, num_nodes))     

In [ ]:
path_length = (higher_adj_mat != 0).argmax(axis = 2)

In [ ]:
plt.title("Path Lengths")
plt.xlabel("Hop Distance")
plt.ylabel("Frequency")
plt.hist(path_length.flatten(),bins=list(range(0,14)))
plt.show()

In [ ]:
target_indices = list(interactions_2['Target'])

In [ ]:
source_indices = list(interactions_2['Source'])

In [ ]:
bigG.add_nodes_from(target_indices, bipartite=0)
bigG.add_nodes_from(source_indices, bipartite=1)

In [ ]:
print(bigG.nodes[5]['bipartite'])
print(bigG.nodes[6]['bipartite'])

In [ ]:
from networkx.algorithms import bipartite

In [ ]:
rect_adj_mat = bipartite.biadjacency_matrix(bigG, source_indices).toarray()